In [1]:
library(dplyr)
library(tidyr)
library(readxl)
library(purrr)
library(stringr)



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
#(a)1

file1 <- "01_data/semester_dummy/semester_data_1.csv"
file2 <- "01_data/semester_dummy/semester_data_2.csv"

semester_dummy_1 <- read.csv(file1)
semester_dummy_2 <- read.csv(file2)

In [3]:
#2・3

#型を揃える
semester_dummy_2 <- semester_dummy_2 %>%
  mutate(across(where(is.integer), as.character))

semester_dummy <- bind_rows(semester_dummy_1, semester_dummy_2)

#1行目を列名に変換
colnames(semester_dummy) <- as.character(unlist(semester_dummy[1, ]))
semester_dummy <- semester_dummy[-1, ]

In [4]:
#4

semester_dummy <- semester_dummy %>% select(-Y)

In [5]:
#5

semester_dummy <- semester_dummy %>%
  arrange(unitid, year) %>%
  group_by(unitid) %>%
  mutate(
    prev_quarter = lag(quarter, default = first(quarter)),
    first_quarter = first(quarter),
    first_year = first(year),
    transition_year_temp = case_when(
      first_quarter == 0 & year == first_year ~ as.character(first_year),
      prev_quarter == 1 & quarter == 0 ~ as.character(year),
      TRUE ~ NA_character_
    ),
    transition_year = na.omit(transition_year_temp)[1]
  ) %>%
  fill(transition_year, .direction = "downup") %>%
  ungroup() %>%
  select(-prev_quarter, -first_quarter, -first_year, -transition_year_temp)

In [6]:
#6

semester_dummy <- semester_dummy %>%
  mutate(
    introduction = ifelse(!is.na(transition_year) &
                            year >= transition_year, 1, 0)
  )

In [7]:
#(b)1

#ディレクトリ内の全てのファイル名を取得
file_path <- "01_data/outcome/"
files <- list.files(path = file_path, pattern = "*.xlsx", full.names = TRUE)

#各ファイルの年度情報を取得し、読み込んでリストに格納
outcome_list <- map(files, function(file) {
  #年度をファイル名から抽出
  year <- as.numeric(gsub(".*([0-9]{4}).xlsx", "\\1", file))
  #ファイルを読み込み
  data <- read_excel(file)
  #年度情報を追加
  data <- mutate(data, year = year)
  return(data)
})

outcome <- bind_rows(outcome_list)

In [8]:
#2

outcome <- outcome %>%
  mutate(women_gradrate_4yr_scale = women_gradrate_4yr * 0.01)

In [9]:
#3・4

#chrのデータを数値に変換
outcome <- outcome %>%
  mutate(across(where(is.character), as.numeric))

#新しい変数を作成し、有効数字3桁に設定
outcome <- outcome %>%
  mutate(
    women_gradrate_4yr_scale = round(women_gradrate_4yr * 0.01, 3),
    men_gradrate_4yr = round((m_4yrgrads / m_cohortsize) * 100, 3),
    total_gradrate_4yr = round((tot4yrgrads / totcohortsize) * 100, 3)
  )

In [10]:
#5

outcome <- outcome %>%
  filter(year >= 1991 & year <= 2010)

In [11]:
#(c)1

covariates <- read_excel("01_data/covariates.xlsx")

In [12]:
covariates <- covariates %>%
  #2
  rename(unitid = university_id) %>%
  #3
  mutate(unitid = str_replace_all(unitid, "aaaa", "")) %>%
  #4
  pivot_wider(names_from = category, values_from = value)

In [13]:
#5

outcome_year_range <- range(outcome$year, na.rm = TRUE)
semester_dummy_year_range <- range(semester_dummy$year, na.rm = TRUE)

covariates <- covariates %>%
  filter(year >= min(outcome_year_range[1], semester_dummy_year_range[1]) &
           year <= max(outcome_year_range[2], semester_dummy_year_range[2]))


In [14]:
#6

outcome_unitids <- unique(outcome$unitid)
covariates <- covariates %>%
  filter(unitid %in% outcome_unitids)

In [15]:
#(d)

#数値型に変換
covariates <- covariates %>%
  mutate(unitid = as.numeric(unitid), year = as.numeric(year))
semester_dummy <- semester_dummy %>%
  mutate(unitid = as.numeric(unitid), year = as.numeric(year))
outcome <- outcome %>%
  mutate(unitid = as.numeric(unitid), year = as.numeric(year))

master <- covariates %>%
  full_join(semester_dummy, by = c("unitid", "year")) %>%
  full_join(outcome, by = c("unitid", "year"))

write.csv(master, "master.csv", row.names = TRUE)